In [9]:
#1
#only mauval input given to file
#note that 1st letter is always upper case and rest of all are lowercase letters
src_name = "Hyderabad"
dst_name = "Goa"
#delay_days=1
next_day = "20-Jun-2024"



import requests
import pandas as pd
import datetime
import time
import shlex
import os
import json
import numpy as np
import csv
from datetime import datetime as dt


# List of cities with their IDs
cities = {
    "Mumbai": 462,
    "Hyderabad": 124,
    "Bangalore": 122,
    "Chennai": 123,
    "Vizag": 248,
    "Vijayawada": 134,
    "Guntur": 137,
    "Tirupati": 71756,
    "Madurai": 126,
    "Coimbatore": 141,
    "Trichy": 71929,
    "Pondicherry": 233,
    "Pune": 130,
    "Indore": 313,
    "Nagpur": 624,
    "Goa": 210,
    "Warangal": 95479,
    "Nellore": 131,
    "Kochi": 216,
    "Dharwad": 167,
    "Mysore": 129,
    "Mangaluru": 95222,
    "Thiruvananthapuram": 71425
}

#long_path = r"\\?\C:\very\long\path\to\your\file.txt"


#next_day = (datetime.datetime.now() + datetime.timedelta(days=delay_days)).strftime('%d-%b-%Y')
main_input_dir=fr"\\?\C:\Users\T VIJAYA BALAJI\Desktop\final_data_collection\{src_name}_to_{dst_name}\data_on_{next_day}\buses_on_{next_day}"
#os.makedirs(main_input_dir,exist_ok=True) ===> there is no need to check as we will run after creating this dir by fetching_seatdata.ipynb file,so if we get any error about existence then it implies we havent collected data on that day

next_day




'20-Jun-2024'

In [21]:
#2
main_output_dir = fr"\\?\C:\Users\T VIJAYA BALAJI\Desktop\final_data_collection\{src_name}_to_{dst_name}\data_on_{next_day}\summary_on_{next_day}"
os.makedirs(main_output_dir,exist_ok=True)

# Function to get the hour from folder name
def get_hour_from_foldername(foldername):
    parts = foldername.split('_')
    hour_part = parts[-1]
    hour_part = dt.strptime(hour_part, "%I%p,%d-%m-%Y")
    return dt.strptime(hour_part.strftime("%I%p,%d-%m-%Y"),"%I%p,%d-%m-%Y")






# Function to get the prefix before the second underscore  for getting travelsname_operatorid_routeid eg:ANAGHA TRAVELS_26880_25731868_Hyderabad_Tirupati_seat_data_on_01.45PM,18-06-2024_at_01PM
def get_prefix(filename):
    parts = filename.split('_')
    return parts[0] + '_' + parts[1]+'_'+parts[2]





# List all folders and sort them by hour
folders = [f for f in os.listdir(main_input_dir) if os.path.isdir(os.path.join(main_input_dir, f))]
#def get_creation_time(file):
#    return dt.fromtimestamp(os.path.getctime(os.path.join(main_input_dir, file)))
#folders.sort(key=get_creation_time)
folders.sort(key=get_hour_from_foldername)

for i in range(len(folders) - 1):
    input_dir_1 = os.path.join(main_input_dir, folders[i])
    input_dir_2 = os.path.join(main_input_dir, folders[i + 1])
    
    hour1 = get_hour_from_foldername(folders[i])
    hour2 = get_hour_from_foldername(folders[i + 1])
    
    # List all files in both directories
    files_1 = [f for f in os.listdir(input_dir_1) if f.endswith('.csv')]
    files_2 = [f for f in os.listdir(input_dir_2) if f.endswith('.csv')]
    
    # Get prefixes for files
    prefixes_1 = {get_prefix(f): f for f in files_1}
    prefixes_2 = {get_prefix(f): f for f in files_2}

    # Ensure both directories have the same prefixes
    common_prefixes = set(prefixes_1.keys()).intersection(set(prefixes_2.keys()))

    # Process each common file
    for prefix in common_prefixes:

        file1 = prefixes_1[prefix]
        file2 = prefixes_2[prefix]
        
        file_path1 = os.path.join(input_dir_1, file1)
        file_path2 = os.path.join(input_dir_2, file2)

        headers = {
           'SEATS': [],
           'STATUS': [],
           'BOOKING TIME': [],
           'REVENUE COLLECTED(only base fare)': []
                   }
        total_df=pd.DataFrame(headers)
        travels_name = prefix.split('_')[0]
        operator_id=prefix.split('_')[1]
        route_id=prefix.split('_')[2]
        summary_filename = f"{travels_name}_{src_name}_to_{dst_name}_data_on_{next_day}_{operator_id}_{route_id}.csv"
        summuary_filepath = os.path.join(main_output_dir,summary_filename)
        file_exists = os.path.isfile(summuary_filepath)
        if not file_exists:
            total_df.to_csv(summuary_filepath,index=False)



        # Load the CSV files
        try :
            df1 = pd.read_csv(file_path1)
        except Exception as e:
            print(f"Error reading {file_path1}: {e}")
            continue
        try :
            df2 = pd.read_csv(file_path2)
        except Exception as e:
            print(f"Error reading {file_path2}: {e}")
            continue    
        #df2 = pd.read_csv(file_path2)
        df1 = df1[:-1]
        df2= df2[:-1]
        # Sort data by 'Id'
        df1['id']=df1['seat']
        df2['id']=df2['seat']
        df1.set_index('id',inplace=True)
        df2.set_index('id',inplace=True)

        df1['occupied'] = pd.to_numeric(df1['occupied'], errors='coerce')
        df1['revenue'] = pd.to_numeric(df1['revenue'], errors='coerce')
        df2['occupied'] = pd.to_numeric(df2['occupied'], errors='coerce')
        df2['revenue'] = pd.to_numeric(df2['revenue'], errors='coerce')

        merged_df = pd.merge(df1, df2, on='seat', suffixes=(f'_on_{hour1}', f'_on_{hour2}'))
        merged_df['id'] = merged_df['seat']
        merged_df.set_index('id',inplace=True)
        merged_df[f'occupied_between_{hour1}_and_{hour2}'] = (df2['occupied'] - df1['occupied']).fillna (0).astype(int)
        #this line is taken if we have dataset instaed of column for  merged_df[f'Fares_Amount_on_{hour1}'] 
        merged_df[f'Fares_Amount_on_{hour1}'] = merged_df[f'Fares_Amount_on_{hour1}'].apply(pd.to_numeric, errors='coerce')
        merged_df[f'occupied_between_{hour1}_and_{hour2}'] = pd.to_numeric(merged_df[f'occupied_between_{hour1}_and_{hour2}'], errors='coerce') 
        
        merged_df[f'revenue_on_seat_between_{hour1}_and_{hour2}'] = merged_df.apply(
                 lambda row: (row[f'Fares_Amount_on_{hour1}'] if row[f'occupied_between_{hour1}_and_{hour2}'] == 1
                 else (-1*row[f'Fares_Amount_on_{hour1}'] if row[f'occupied_between_{hour1}_and_{hour2}'] == -1 else 0)), axis=1)
#        merged_df= merged_df.drop([f'Fares_Amount_on_{hour1}',f'Fares_Amount_on_{hour2}',f'revenue_on_{hour1}',f'revenue_on_{hour2}',f'Is_Available_on_{hour1}',f'Is_Available_on_{hour2}',f'occupied_on_{hour1}',f'occupied_on_{hour2}'],axis=1)
#        last_rows=pd.DataFrame({
#        'Id':[f'Total seat filled between {hour1} and {hour2}',f'Total revenue between {hour1} and {hour2}',f'Cancelled seats between {hour1} and {hour2}'],
#        f'occupied_between_{hour1}_and_{hour2}':[merged_df[f'occupied_between_{hour1}_and_{hour2}'].astype(float).sum(),merged_df[f'revenue_on_seat_between_{hour1}_and_{hour2}'].astype(float).sum(),merged_df[f'occupied_between_{hour1}_and_{hour2}'].astype(float).eq(-1).sum()],
#        f'revenue_on_seat_between_{hour1}_and_{hour2}':["","",""]
  
        for index, row in merged_df.iterrows():
            if row[f'occupied_between_{hour1}_and_{hour2}'] == 1:
                total_df_row = pd.DataFrame({
                    'SEATS': [row['seat']],
                    'STATUS': ["CONFIRMED"],
                    'BOOKING TIME':[f"{hour1} to {hour2}"],
                    'REVENUE COLLECTED(only base fare)':[row[ f'revenue_on_seat_between_{hour1}_and_{hour2}']]
                    })
                total_df_row.to_csv(summuary_filepath, mode='a', index=False, header=False)
#            elif row[f'occupied_between_{hour1}_and_{hour2}'] == -1:
#                total_df_row = pd.DataFrame({
#                    'SEATS': [row['seat']],
#                    'STATUS': ["CANCELLED"],
#                    'BOOKING TIME':[f"{hour1} to {hour2}"],
#                    'REVENUE COLLECTED(only base fare)':[row[ f'revenue_on_seat_between_{hour1}_and_{hour2}']]
#                    })
#                total_df_row.to_csv(summuary_filepath, mode='a', index=False, header=False)
            else: 
                pass         

#        merged_df = pd.concat([merged_df, last_rows], ignore_index=True)
        
print(f"sucessesfully completed saving comparisions summary of all buses in route {src_name} to {dst_name} on {next_day}")        


sucessesfully completed saving comparisions summary of all buses in route Hyderabad to Goa on 20-Jun-2024


In [25]:
type(hour1)

datetime.datetime

In [22]:

folders = [f for f in os.listdir(main_input_dir) if os.path.isdir(os.path.join(main_input_dir, f))]
# Function to get the hour from folder name
def get_hour_from_foldername(foldername):
    parts = foldername.split('_')
    hour_part = parts[-1]
    hour_part = dt.strptime(hour_part, "%I%p,%d-%m-%Y")
    return dt.strptime(hour_part.strftime("%I%p,%d-%m-%Y"),"%I%p,%d-%m-%Y")
folders.sort(key=get_hour_from_foldername)

for f in folders:
    print(get_hour_from_foldername(f).strftime("%I.%M%p,%d-%m-%Y")+"  "+f)

05.00PM,18-06-2024  buses_on_20-Jun-2024_at_05PM,18-06-2024
06.00PM,18-06-2024  buses_on_20-Jun-2024_at_06PM,18-06-2024
07.00PM,18-06-2024  buses_on_20-Jun-2024_at_07PM,18-06-2024
08.00PM,18-06-2024  buses_on_20-Jun-2024_at_08PM,18-06-2024
09.00PM,18-06-2024  buses_on_20-Jun-2024_at_09PM,18-06-2024
11.00PM,18-06-2024  buses_on_20-Jun-2024_at_11PM,18-06-2024
12.00AM,19-06-2024  buses_on_20-Jun-2024_at_12AM,19-06-2024
01.00AM,19-06-2024  buses_on_20-Jun-2024_at_01AM,19-06-2024
02.00AM,19-06-2024  buses_on_20-Jun-2024_at_02AM,19-06-2024
03.00AM,19-06-2024  buses_on_20-Jun-2024_at_03AM,19-06-2024
04.00AM,19-06-2024  buses_on_20-Jun-2024_at_04AM,19-06-2024
05.00AM,19-06-2024  buses_on_20-Jun-2024_at_05AM,19-06-2024
06.00AM,19-06-2024  buses_on_20-Jun-2024_at_06AM,19-06-2024
07.00AM,19-06-2024  buses_on_20-Jun-2024_at_07AM,19-06-2024
08.00AM,19-06-2024  buses_on_20-Jun-2024_at_08AM,19-06-2024
09.00AM,19-06-2024  buses_on_20-Jun-2024_at_09AM,19-06-2024
10.00AM,19-06-2024  buses_on_20-Jun-2024

In [23]:
#3 creating final rows in summary files
summary_files_dir = fr"C:\Users\T VIJAYA BALAJI\Desktop\final_data_collection\{src_name}_to_{dst_name}\data_on_{next_day}\summary_on_{next_day}"
summary_files = [f for f in os.listdir(summary_files_dir) if f.endswith('.csv')]

for file in summary_files:
        file_path= os.path.join(summary_files_dir,file)
        total_df=pd.read_csv(file_path)
        total_df = total_df.drop_duplicates(subset=['SEATS'], keep='last')
        total_df.to_csv(file_path, index=False) 

#        total_summary_rows=pd.DataFrame({
#            'SEATS': ["TOTAL SEATS OCCUPIED","TOTAL SEATS CANCELLED","TOTAL REVENUE COLLECTED"],
#            'STATUS': [total_df['STATUS'].value_counts().get('CONFIRMED', 0),total_df['STATUS'].value_counts().get('CANCELLED', 0),total_df['REVENUE COLLECTED(only base fare)'].astype(float).sum()],
#            'BOOKING TIME':[f"","",""],
#            'REVENUE COLLECTED(only base fare)':["","",""]
#        
#        })

        total_summary_rows=pd.DataFrame({
            'SEATS': ["TOTAL SEATS OCCUPIED","TOTAL REVENUE COLLECTED"],
            'STATUS': [total_df['STATUS'].value_counts().get('CONFIRMED', 0),total_df['REVENUE COLLECTED(only base fare)'].astype(float).sum()],
            'BOOKING TIME':[f"",""],
            'REVENUE COLLECTED(only base fare)':["",""]
        
        })

        total_summary_rows.to_csv(file_path, mode='a', index=False, header=False)
        print(total_df)


Empty DataFrame
Columns: [SEATS, STATUS, BOOKING TIME, REVENUE COLLECTED(only base fare)]
Index: []
  SEATS     STATUS                                BOOKING TIME  \
0    L4  CONFIRMED  2024-06-20 12:00:00 to 2024-06-20 13:00:00   
1    L3  CONFIRMED  2024-06-20 12:00:00 to 2024-06-20 13:00:00   
2    U5  CONFIRMED  2024-06-20 12:00:00 to 2024-06-20 13:00:00   

   REVENUE COLLECTED(only base fare)  
0                            1153.95  
1                            1153.95  
2                            1101.45  
   SEATS     STATUS                                BOOKING TIME  \
0     4L  CONFIRMED  2024-06-19 18:00:00 to 2024-06-19 19:00:00   
1     6U  CONFIRMED  2024-06-20 12:00:00 to 2024-06-20 13:00:00   
2     5U  CONFIRMED  2024-06-20 12:00:00 to 2024-06-20 13:00:00   
3     4U  CONFIRMED  2024-06-20 12:00:00 to 2024-06-20 13:00:00   
4     1U  CONFIRMED  2024-06-20 15:00:00 to 2024-06-20 17:00:00   
5     3L  CONFIRMED  2024-06-20 15:00:00 to 2024-06-20 17:00:00   
6     6L  